In [1]:
# 104_job_bank with Selenium (Overview)
# import packages
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import csv , time ,sys

def create_driver():
    service = Service("./chromedriver")
    chrome_options = Options()
    chrome_options.page_load_strategy = 'eager'
    driver = Chrome(service=service,options = chrome_options)
    return driver

# Get Driver
driver = create_driver()


In [122]:
# define function : get url and write into csv
def access_detail(nth,job_link):
    print(f'===============這是第{nth}個工作 ===================')
    print(f'========== 工作連結為{job_link} ================')
    # service = Service("./chromedriver")
    # chrome_options = Options()
    # chrome_options.page_load_strategy = 'eager'
    driver2 = create_driver()
    driver2.execute_async_script(
        """
    count = 400;
    let callback = arguments[arguments.length - 1];
    t = setTimeout(function scrolldown(){
        console.log(count, t);
        window.scrollTo(0, count);
        if(count < (document.body.scrollHeight || document.documentElement.scrollHeight)){
        count+= 400;
        t = setTimeout(scrolldown, 1000);
        }else{
        callback((document.body.scrollHeight || document.documentElement.scrollHeight));
        }
    }, 1000);"""
    )
    
    # get job's info
    company_name= driver2.find_element(By.CLASS_NAME, "btn-link.t3.mr-6").text
    
    time.sleep(3)
    driver2.close()

In [123]:
# 104 home page's url
url = "https://www.104.com.tw/jobs/main/"  # go to jobbank
driver.get(url) # surf jobbank with browser
driver.maximize_window() # maximize broswer window -> get more HTML ASAP
time.sleep(3)

In [124]:
# KeyWord of interest
driver.find_element(by=By.XPATH, value='/html/body/article[1]/div/div/div[4]/div/input').send_keys("Python工程師")
time.sleep(3)

In [125]:
# Push search button
driver.find_element(by=By.XPATH,value='/html/body/article[1]/div/div/div[4]/div/button').click()
time.sleep(3)

In [126]:
# Scroll action
# Step 1 : get all pages' count
button = driver.find_element(By.CLASS_NAME,"b-clear-border.page-select.js-paging-select.gtm-paging-bottom")
Total_pages_count = len(button.text.split("\n"))

# Step 2 : auto scroll + manual click
manual_scroll_count = 1
request_nth = input("請問要搜尋幾個職缺? :")
first_counts = len(driver.find_elements(By.CLASS_NAME, "js-job-link")) # get counts of 1st page's job

# Step 3 : create job_table in beggining
job_table = pd.DataFrame() # create empty table

# Step 4 : crawling info of jobs and writing it into job_table(DataFrame)
while True :
    if manual_scroll_count + 15 == Total_pages_count:
        print("================ 已到達最底，翻頁完成 !!=======================")
        break
    jobs = driver.find_elements(By.CLASS_NAME, "js-job-link")
    print(f"================ 目前共有{len(jobs)}個職缺 ===================")
    if (len(jobs) - first_counts) / 20 + 1 == 1: # if 1st page
        for nth,job in enumerate(jobs,start = 1):
            driver.execute_script("arguments[0].scrollIntoView();",job) # slide to target elem
            driver.execute_script("window.scrollBy(0, -240);")
            job_link = job.get_attribute("href")
            job_table = save_to_table(job_table,job_link)
            if str(nth) == request_nth:
                print(f"============= 已爬取 {nth} 個職缺，停止服務 !! ===============")
                sys.exit()
    else : # another page
        for nth,job in enumerate(jobs[-20:],start = len(jobs)- (20 + 1)): # get last 20 jobs
            driver.execute_script(f"arguments[0].scrollIntoView();",job)  # slide to target elem
            driver.execute_script("window.scrollBy(0, -240);")
            job_link = job.get_attribute("href")
            save_to_csv(nth,job_link)
            if str(nth) == request_nth:
                print(f"============= 已爬取 {nth} 個職缺，停止服務 !! ===============")
                sys.exit()
    # driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
    try:
        button = driver.find_element(By.CLASS_NAME,"b-btn.b-btn--link.js-more-page")
        driver.execute_script("arguments[0].click();", button)
        print(f"================ 需要手動!! 按一下按鈕! -> 目前翻到第{manual_scroll_count + 15} 頁=================")
        manual_scroll_count += 1
        time.sleep(2)
    except Exception :
        print("================ 不用手動，繼續滑下去!! 直到翻到第15頁 =================")

請問要搜尋幾個職缺? : 29


================目前有24個職缺===================
===============這是第1個工作 ===================
========== 工作連結為https://www.104.com.tw/job/7moyg?jobsource=hotjob_chr ================
===============這是第2個工作 ===================
========== 工作連結為https://www.104.com.tw/job/7uwh4?jobsource=hotjob_chr ================
===============這是第3個工作 ===================
========== 工作連結為https://www.104.com.tw/job/7at0b?jobsource=hotjob_chr ================
===============這是第4個工作 ===================
========== 工作連結為https://www.104.com.tw/job/565k4?jobsource=hotjob_chr ================
===============這是第5個工作 ===================
========== 工作連結為https://www.104.com.tw/job/7vxl0?jobsource=jolist_c_relevance ================
===============這是第6個工作 ===================
========== 工作連結為https://www.104.com.tw/job/7uskr?jobsource=jolist_c_relevance ================
===============這是第7個工作 ===================
========== 工作連結為https://www.104.com.tw/job/6xglu?jobsource=jolist_c_relevance ================
===============這是第8個工作

KeyboardInterrupt: 

In [41]:
len(jobs)
(len(jobs) - first_counts) / 20 + 1

1.0

46

46

In [295]:
driver.execute_script("arguments[0].scrollIntoView();",jobs[4])
 

In [272]:
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", jobs[5])

In [264]:
jobs[2].text

'Frontend / Computer Vision / Golang / iOS Senior Engineer'

In [ ]:
# Scroll Test


In [6]:
# Scroll action
# Step 1 : get all pages' count
button = driver.find_element(By.CLASS_NAME,"b-clear-border.page-select.js-paging-select.gtm-paging-bottom")
Total_pages_count = len(button.text.split("\n"))

# Step 2 : auto scroll + manual click
manual_scroll_count = 1
while manual_scroll_count :
    if manual_scroll_count + 15 == Total_pages_count:
        print("================ 已到達最底，翻頁完成 !!=======================")
        manual_scroll_count = 0
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
    try :
        button = driver.find_element(By.CLASS_NAME,"b-btn.b-btn--link.js-more-page")
        driver.execute_script("arguments[0].click();", button)
        print(f"================需要手動!! 按一下按鈕! -> 目前翻到第{manual_scroll_count + 15}頁=================")
        manual_scroll_count += 1
        time.sleep(2)
    except Exception as e:
        print("================不用手動，繼續滑下去!! 直到翻到第15頁=================")

================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================不用手動，繼續滑下去!! 直到翻到第15頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第16頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第17頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第18頁=================
================需要手動!! 按一下按鈕! -> 目前翻到第19頁============

KeyboardInterrupt: 

In [13]:
job_names = driver.find_elements(By.CLASS_NAME, "js-job-link")
len(job_names)

890

In [14]:
for i,job_name in enumerate(job_names):
    job_link = job_name.get_attribute("href")
    print(i,job_link)

0 https://www.104.com.tw/job/7vbng?jobsource=hotjob_chr
1 https://www.104.com.tw/job/72cz1?jobsource=hotjob_chr
2 https://www.104.com.tw/job/7wbdq?jobsource=hotjob_chr
3 https://www.104.com.tw/job/7tz7f?jobsource=hotjob_chr
4 https://www.104.com.tw/job/7vp6c?jobsource=hotjob_chr
5 https://www.104.com.tw/job/4i43s?jobsource=hotjob_chr
6 https://www.104.com.tw/job/5mqe7?jobsource=hotjob_chr
7 https://www.104.com.tw/job/7a4or?jobsource=hotjob_chr
8 https://www.104.com.tw/job/7al1r?jobsource=hotjob_chr
9 https://www.104.com.tw/job/7chgf?jobsource=hotjob_chr
10 https://www.104.com.tw/job/7vxl0?jobsource=jolist_d_relevance
11 https://www.104.com.tw/job/7uskr?jobsource=jolist_d_relevance
12 https://www.104.com.tw/job/6xglu?jobsource=jolist_d_relevance
13 https://www.104.com.tw/job/7g51e?jobsource=jolist_d_relevance
14 https://www.104.com.tw/job/695ve?jobsource=jolist_d_relevance
15 https://www.104.com.tw/job/7mpui?jobsource=jolist_d_relevance
16 https://www.104.com.tw/job/63ffy?jobsource=joli

In [9]:
# find job details in articles which has same form individually
for nth_job in range(jobs_numbers) :
    # articles = driver.find_elements(By.CLASS_NAME, "b-block--top-bord.job-list-item.b-clearfix.js-job-item")
    # find nth-job's link
    job_name = driver.find_elements(By.CLASS_NAME, "js-job-link")[nth_job]
    # get value of attribute (key) in every element (ex: "href" = "url" 、 "class" = "js-job-link")
    job_link = job_name.get_attribute("href")
    print(job_name.text)
    print(job_link)
    driver.get(job_link)
    time.sleep(3)
    driver.back()
    time.sleep(5)

大流量社群平台資深後端工程師 /Senior Backend Engineer(Golang)
https://www.104.com.tw/job/5bdsg?jobsource=hotjob_chr
PHP Laravel工程師(中階以上)
https://www.104.com.tw/job/7p2s1?jobsource=hotjob_chr
軟體專案管理人員
https://www.104.com.tw/job/74y1s?jobsource=hotjob_chr
Frontend / Computer Vision / Golang / iOS Senior Engineer
https://www.104.com.tw/job/7tz7f?jobsource=hotjob_chr
(總公司-資訊部)後端工程師
https://www.104.com.tw/job/75ytw?jobsource=hotjob_chr
前端與後端技術工程師 (具三年以上經驗)
https://www.104.com.tw/job/7kuen?jobsource=hotjob_chr
高級後端工程師**月薪60,000~80,000**
https://www.104.com.tw/job/74xzl?jobsource=hotjob_chr
資深網站前端工程師 (Senior Frontend Developer)
https://www.104.com.tw/job/5udee?jobsource=hotjob_chr
資深後端工程師**月薪80,000~100,000**
https://www.104.com.tw/job/74y02?jobsource=hotjob_chr
PHP 或 Java後端工程師 (視訊/線上面談)-擴編職缺
https://www.104.com.tw/job/64v7o?jobsource=hotjob_chr
Backend Engineer 後端工程師
https://www.104.com.tw/job/7nq2e?jobsource=jolist_b_relevance
Backend Engineer後端工程師
https://www.104.com.tw/job/5bwie?jobsource=jolist_b_relev

In [10]:
jobs_numbers

30

In [11]:
job_name = driver.find_elements(By.CLASS_NAME, "js-job-link")[29]

In [12]:
job_name.text

'RD-後端工程師 Backend Engineer'

In [ ]:
# find HTML element of jobs' info
# articles = driver.find_elements(By.CLASS_NAME, "b-block--top-bord.job-list-item.b-clearfix.js-job-item")
driver.execute_script(f"window.scrollTo(0, {scroll+240})")  # 下滑下一篇
scroll += 240
jobs = driver.find_elements(By.CLASS_NAME, "js-job-link")
jobs_numbers = len(jobs)
jobs_numbers

In [1]:
# 104_job_bank with Selenium (Overview)
# import packages
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import csv , time ,sys

In [7]:
# 單元測試 : 測試內部標籤如何抓
# I : create job_table in beggining
job_link = 'https://www.104.com.tw/job/7vxl0?jobsource=jolist_d_relevance'
job_table = pd.DataFrame() # create empty table
service = Service("./chromedriver")
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
driver2 = Chrome(service=service,options = chrome_options)
driver2.get(job_link)

In [9]:
# 公司名稱
company_name= driver2.find_element(By.CLASS_NAME, "btn-link.t3.mr-6").text
company_name

'博凡星國際有限公司'

In [10]:
# 工作名稱
job_name = driver2.find_element(By.CLASS_NAME, "pr-6.text-break").text
job_name

'Python工程師 03/02更新'

In [11]:
# 職務類別
job_type = driver2.find_elements(By.CLASS_NAME, "col.p-0.list-row__data")[0].text.replace("\n","")
job_type

'Internet程式設計師、軟韌體測試工程師、軟體設計工程師'

In [12]:
# 工作薪資
job_pay = driver2.find_elements(By.CLASS_NAME, "col.p-0.list-row__data")[1].text
job_pay

'待遇面議 （經常性薪資達 4 萬元或以上）'

In [13]:
# 工作性質 (正職 FULL TIME vs 兼職 PART TIME)
ft_pt = driver2.find_elements(By.CLASS_NAME, "col.p-0.list-row__data")[2].text
ft_pt

'全職'

In [14]:
# 上班地點
job_location = driver2.find_elements(By.CLASS_NAME, "col.p-0.list-row__data")[3].text
job_location

'台中市西屯區潮洋里市政北二路238號15樓之1'

In [15]:
# 工作內容 (500字內)
job_content = driver2.find_element(By.CLASS_NAME, "mb-5.r3.job-description__content.text-break").text
if len(job_content) > 500 :
    job_content = job_content[:500]
    job_content += ".....詳情請見網站"

In [16]:
# 科系要求
subject = driver2.find_elements(By.CLASS_NAME, "col.p-0.list-row__data")[13].text

In [17]:

# 擅長工具
job_tools = driver2.find_elements(By.CLASS_NAME, "col.p-0.list-row__data")[15].text.split('\n')[0]

In [18]:
# 其他要求
another_require = driver2.find_elements(By.CLASS_NAME,"col.p-0.list-row__data")[-1].text

In [19]:
job_content

'工作內容：\n1.集成系統開發與維護\n2.串接第三方服務\n3.客製化系統開發\n\n基本要求：\n-對開發與技術學習有熱忱，且願意不斷挑戰自我\n-熟悉Redis、Mongodb\n-具Web服務(WCF服務/SOAP/Web API)等開發與串接經驗，並具備良好物件導向程式設計能力，及解決問題能力\n-熟悉git版控，並有多人協作經驗\n\n加分項：\n※具一年以上的Python開發實務經驗\n※熟悉遊戲第三方串接相關經驗\n※有使用過RabbitMQ\n※精通NoSQL或任一資料庫\n※有高併發、高可用、分散式服務開發經驗者為佳\n※熟悉並能靈活運用設計模式優化專案架構\n※具大型網站或後端系統架構規劃與設計開發經驗、資料庫規劃能力\n※熟悉部署流程，CI/CD建置\n※熟悉Docker相關技術\n※建置過開發SOP文件、規範、流程"'

In [20]:
each_job_detail = {
                   "company_name" : company_name,
                   "job_name" : job_name,
                   "job_type" : job_type,
                   "job_pay" : job_pay,
                   "ft_pt" : ft_pt,
                   "job_location" : job_location,
                   "subject" : subject,
                   "job_tools" : job_tools,
                   "job_content" : job_content,
                   "another_require" : another_require
}
each_job_table = pd.DataFrame([each_job_detail])

[{'company_name': '緯創軟體股份有限公司',
  'job_name': 'T-【2023軟體技術人才招募】Python 工程師_人才招募 03/03更新',
  'job_type': '軟體設計工程師、其他資訊專業人員、Internet程式設計師',
  'job_pay': '待遇面議 （經常性薪資達 4 萬元或以上）',
  'ft_pt': '全職',
  'job_location': '新北市汐止區新竹或台北',
  'job_content': '【2年以上軟體相關經驗者，歡迎加入我們行列!】\n\n【工作內容】\n1. 熟悉Python程式語言，負責後端開發\n2. 有 Django 開發大型網站兩年以上的經驗\n3. 有 JQuery、JavaScript and Bootstrap 的網頁開發技術和兩年以上經驗 \n4. 有 database 兩年以上經驗(PostgreSQL、MySQL、Redis、MongoDB…)\n5. 熟悉git操作\n加分項目:\n1. 具有Docker 和 Kubernetes 的經驗\n2. 熟悉 RDBMS 或 NoSQL 數據庫\n3. 在 DevOps 實踐中工作的經驗\n5. 有參加過產品專案開發的完整經驗\n6. 有能力預估所賦予工作所需的開發時程\n\n【主要產業】\n銀行/保險/證券/半導體/面板/電子/通訊業...等系統開發\n\n【工作地點】\n\n固定地點:包括大台北地區(雙北市)、新竹科學園區、台中(台中市/台中科學園區)、台南科學園區、高雄市等區域擇一安排\n\n【薪資】\n面議(依經驗與能力核薪)\n\n緯創軟體是台灣唯一專注於全球交付的資訊服務上櫃公司。因應業績不斷成長，需要大量人才加入國際軟體開發團隊。\n\n因...詳情請見網站'},
 {'company_name': '緯創軟體股份有限公司',
  'job_name': 'T-【2023軟體技術人才招募】Python 工程師_人才招募 03/03更新',
  'job_type': '軟體設計工程師、其他資訊專業人員、Internet程式設計師',
  'job_pay': '待遇面議 （經常性薪資達 4 萬元或以上）',
  'ft_pt': '全職',
  'job_location': '新北市汐止區

In [21]:
job_table = pd.concat([job_table,each_job_table], ignore_index=True)
job_table

,company_name,job_name,job_type,job_pay,ft_pt,job_location,subject,job_tools,job_content,another_require
0,博凡星國際有限公司,Python工程師 03/02更新,Internet程式設計師、軟韌體測試工程師、軟體設計工程師,待遇面議 （經常性薪資達 4 萬元或以上）,全職,台中市西屯區潮洋里市政北二路238號15樓之1,不拘,Linux、Python、MySQL,工作內容：\n1.集成系統開發與維護\n2.串接第三方服務\n3.客製化系統開發\n\n基本...,未填寫


In [22]:
display(job_table)

,company_name,job_name,job_type,job_pay,ft_pt,job_location,subject,job_tools,job_content,another_require
0,博凡星國際有限公司,Python工程師 03/02更新,Internet程式設計師、軟韌體測試工程師、軟體設計工程師,待遇面議 （經常性薪資達 4 萬元或以上）,全職,台中市西屯區潮洋里市政北二路238號15樓之1,不拘,Linux、Python、MySQL,工作內容：\n1.集成系統開發與維護\n2.串接第三方服務\n3.客製化系統開發\n\n基本...,未填寫


In [23]:
job_table.to_csv("test.csv",index=False) # 不顯示索引值並儲存至csv

In [61]:
import pandas as pd

# 建立 DataFrame，index 設為 None
df = pd.DataFrame({'名稱': ['張三', '李四', '王五'], '年齡': [20, 25, 30]}, index=None)

# 使用 to_string() 方法顯示 DataFrame，設定 index=False 可以移除最左邊的索引值
print(df.to_string(index=False))

名稱  年齡
張三  20
李四  25
王五  30
